# [Unicode 文字列](https://www.tensorflow.org/tutorials/load_data/unicode)

Unicodeはほぼ全ての言語の文字を表すことができ、それぞれの文字は`0`から`0x10FFFF`までの整数コードに割り振られている。

ここでは、Unicode文字列をTensorFlowで扱ってみる。

In [1]:
import tensorflow as tf

TensorFlowでは、デフォルトでUnicodeはutf-8を用いて暗号化する。

In [2]:
tf.constant(u"Thanks 😊")

<tf.Tensor: shape=(), dtype=string, numpy=b'Thanks \xf0\x9f\x98\x8a'>

`tf.string`は長さの違うバイト文字列を保持することができる。したがって、文字列の長さはテンソルの次元には含まれない。

In [3]:
tf.constant([u"You're", u"welcome!"]).shape

TensorShape([2])

## Unicode を表現する

TensorFlowでUnicode文字列を表現するには、以下の二つの基本的な方法がある：

- `string` scalar：[character encoding](https://en.wikipedia.org/wiki/Character_encoding)を使ってバイト列を暗号化する。
- `int32` vector：それぞれの要素が一つのコードを表す。

In [4]:
# Unicode string, represented as a UTF-8 encoded string scalar.
text_utf8 = tf.constant(u"言語処理")
text_utf8

<tf.Tensor: shape=(), dtype=string, numpy=b'\xe8\xa8\x80\xe8\xaa\x9e\xe5\x87\xa6\xe7\x90\x86'>

In [5]:
# Unicode string, represented as a UTF-16-BE encoded string scalar.
text_utf16be = tf.constant(u"言語処理".encode("UTF-16-BE"))
text_utf16be

<tf.Tensor: shape=(), dtype=string, numpy=b'\x8a\x00\x8a\x9eQ\xe6t\x06'>

In [6]:
# Unicode string, represented as vector of Unicode code points.
text_chars = tf.constant([ord(char) for char in u"言語処理"])
text_chars

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([35328, 35486, 20966, 29702], dtype=int32)>

TensorFlowは次の三つの異なる表現方法間で変換が可能である：

- `tf.strings.unicode_decode`：暗号化された文字列スカラーをコードポイントのベクトルに変換
- `tf.strings.unicode_encode`：コードポイントのベクトルを暗号化された文字列スカラーに変換
- `tf.strings.unicode_transcode`：暗号化された文字列スカラーを他の暗号化手法に変換

In [7]:
tf.strings.unicode_decode(text_utf8,
                          input_encoding='UTF-8')

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([35328, 35486, 20966, 29702], dtype=int32)>

In [8]:
tf.strings.unicode_encode(text_chars,
                          output_encoding='UTF-8')

<tf.Tensor: shape=(), dtype=string, numpy=b'\xe8\xa8\x80\xe8\xaa\x9e\xe5\x87\xa6\xe7\x90\x86'>

In [9]:
tf.strings.unicode_transcode(text_utf8,
                            input_encoding='UTF-8',
                            output_encoding='UTF-16-BE')

<tf.Tensor: shape=(), dtype=string, numpy=b'\x8a\x00\x8a\x9eQ\xe6t\x06'>

複数の文字列をデコードする時、それぞれの文字列の長さは等しくない場合がある。バッチした時`tf.RaggedTensor`が返され、最下層の次元は文字列の長さによって決まる。

In [10]:
# A batch of Unicode strings, each represented as a UTF8-encoded string.
batch_utf8 = [s.encode('UTF-8') for s in
              [u'hÃllo',  u'What is the weather tomorrow',  u'Göödnight', u'😊']]
batch_chars_ragged = tf.strings.unicode_decode(batch_utf8,
                                               input_encoding='UTF-8')
for sentence_chars in batch_chars_ragged.to_list():
    print(sentence_chars)

[104, 195, 108, 108, 111]
[87, 104, 97, 116, 32, 105, 115, 32, 116, 104, 101, 32, 119, 101, 97, 116, 104, 101, 114, 32, 116, 111, 109, 111, 114, 114, 111, 119]
[71, 246, 246, 100, 110, 105, 103, 104, 116]
[128522]


この`tf.RaggedTensor`を直接使うこともできるし、`tf.RaggedTensor.to_tensor`メソッドを使ってパディングされた密な`tf.Tensor`に変換することもできるし、`tf.RaggedTensor.to_sparse`メソッドを使って`tf.SparseTensor`に変換することができる。

In [11]:
batch_chars_padded = batch_chars_ragged.to_tensor(default_value=-1)
print(batch_chars_padded.numpy())

[[   104    195    108    108    111     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]
 [    87    104     97    116     32    105    115     32    116    104
     101     32    119    101     97    116    104    101    114     32
     116    111    109    111    114    114    111    119]
 [    71    246    246    100    110    105    103    104    116     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]
 [128522     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]]


In [12]:
batch_chars_sparse = batch_chars_ragged.to_sparse()
print(batch_chars_sparse)

SparseTensor(indices=tf.Tensor(
[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 0  3]
 [ 0  4]
 [ 1  0]
 [ 1  1]
 [ 1  2]
 [ 1  3]
 [ 1  4]
 [ 1  5]
 [ 1  6]
 [ 1  7]
 [ 1  8]
 [ 1  9]
 [ 1 10]
 [ 1 11]
 [ 1 12]
 [ 1 13]
 [ 1 14]
 [ 1 15]
 [ 1 16]
 [ 1 17]
 [ 1 18]
 [ 1 19]
 [ 1 20]
 [ 1 21]
 [ 1 22]
 [ 1 23]
 [ 1 24]
 [ 1 25]
 [ 1 26]
 [ 1 27]
 [ 2  0]
 [ 2  1]
 [ 2  2]
 [ 2  3]
 [ 2  4]
 [ 2  5]
 [ 2  6]
 [ 2  7]
 [ 2  8]
 [ 3  0]], shape=(43, 2), dtype=int64), values=tf.Tensor(
[   104    195    108    108    111     87    104     97    116     32
    105    115     32    116    104    101     32    119    101     97
    116    104    101    114     32    116    111    109    111    114
    114    111    119     71    246    246    100    110    105    103
    104    116 128522], shape=(43,), dtype=int32), dense_shape=tf.Tensor([ 4 28], shape=(2,), dtype=int64))


長さの等しい複数の文字列を暗号化する時、`tf.Tensor`がインプットとして使われる。

In [13]:
tf.strings.unicode_encode(
    [[99, 97, 116], [100, 111, 103], [ 99, 111, 119]],
    output_encoding='UTF-8'
)

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'cat', b'dog', b'cow'], dtype=object)>

長さの異なる複数の文字列を暗号化する時、`tf.RaggedTensor`を入力する必要がある。

In [14]:
tf.strings.unicode_encode(batch_chars_ragged, output_encoding='UTF-8')

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

パディングされた文字列またはスパースな文字列を扱う場合は、`unicode_encode`を呼び出す前に`tf.RaggedTensor`に変換する。

In [15]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_sparse(batch_chars_sparse),
    output_encoding='UTF-8'
)

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

In [16]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_tensor(batch_chars_padded, padding=-1),
    output_encoding='UTF-8'
)

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

## Unicode 操作

文字列の長さは`tf.strings.length`を使って求める。引数`unit`を指定することで、暗号化された`string`の文字列を正しくカウントできる（デフォルトで`"BYTE"`、そのほかに`"UTF8_CHAR"`と`"UTF16_CHAR"`を渡すことができる）。

In [17]:
# Note that the final character takes up 4 bytes in UTF8.
thanks = u'Thanks 😊'.encode('UTF-8')
num_bytes = tf.strings.length(thanks).numpy()
num_chars = tf.strings.length(thanks, unit='UTF8_CHAR').numpy()
print('{} bytes; {} UTF-8 characters'.format(num_bytes, num_chars))

11 bytes; 8 UTF-8 characters


部分文字列は`tf.strings.substr`を使って求める。`tf.strings.length`同様、`unit`引数を受け取る。

In [18]:
# default: unit='BYTE'. With len=1, we rturn a single byte.
tf.strings.substr(thanks, pos=7, len=1).numpy()

b'\xf0'

In [19]:
# Specifying unit='UTF8_CHAR', we return a single character, which in this case is 4 bytes.
print(tf.strings.substr(thanks, pos=7, len=1, unit='UTF8_CHAR').numpy())

b'\xf0\x9f\x98\x8a'


`tf.strings.unicode_split`はUnicode文字列を一文字ずつに分割する。

In [20]:
tf.strings.unicode_split(thanks, 'UTF-8').numpy()

array([b'T', b'h', b'a', b'n', b'k', b's', b' ', b'\xf0\x9f\x98\x8a'],
      dtype=object)

文字列テンソルを整列するためにそれぞれの文字が始まる位置のオフセットを得る。`tf.strings.unicode_decode_with_offsets`は`unicode_decode`とほぼ同じで、加えてオフセットを二つ目のテンソルとして返す。

In [21]:
codepoints, offsets = tf.strings.unicode_decode_with_offsets(u"🎈🎉🎊", 'UTF-8')

for (codepoint, offset) in zip(codepoints.numpy(), offsets.numpy()):
    print("At byte offset {}: codepoint {}".format(offset, codepoint))

At byte offset 0: codepoint 127880
At byte offset 4: codepoint 127881
At byte offset 8: codepoint 127882


## Unicode スクリプト

それぞれのユニコードのコードポイントはスクリプトという集合に含まれる。TensorFlowでは`tf.strings.unicode_script`メソッドを使って、どのスクリプトを与えられたコードポイントが利用しているかを特定できる。スクリプトのコードは整数値で、[International Components for Unicode (ICU)](http://site.icu-project.org/home)値`UScriptCode`に対応する。

In [22]:
uscript = tf.strings.unicode_script([33464, 1041])  # ['芸', 'Б']

print(uscript.numpy())  # [17, 8] == [USCRIPT_HAN, USCRIPT_CYRILLIC]

[17  8]


In [23]:
# The `tf.strings.unicode_script` operation can also be applied to multidimensional `tf.Tensor`s
# or `tf.RaggedTensor`s of codepoints:
print(tf.strings.unicode_script(batch_chars_ragged))

<tf.RaggedTensor [[25, 25, 25, 25, 25], [25, 25, 25, 25, 0, 25, 25, 0, 25, 25, 25, 0, 25, 25, 25, 25, 25, 25, 25, 0, 25, 25, 25, 25, 25, 25, 25, 25], [25, 25, 25, 25, 25, 25, 25, 25, 25], [0]]>


## 例：単純なセグメンテーション

ここでのセグメンテーションは単語単位にテキストを分割すること。スクリプトの変化を使って単語の境を近似的に求めることができる。

In [24]:
# dtype: string; shape: [num_sentences]
#
# The sentences to process.  Edit this line to try out different inputs!
sentence_texts = [u'Hello, world.', u'世界こんにちは']

まずはじめに、文章を文字コードポイントにデコードし、それぞれの文字についてスクリプトの識別子を見つける。

In [25]:
# dtype: int32; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_codepoint[i, j] is the codepoint for the j'th character in
# the i'th sentence.
sentence_char_codepoint = tf.strings.unicode_decode(sentence_texts, 'UTF-8')
print(sentence_char_codepoint)

# dtype: int32; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_scripts[i, j] is the unicode script of the j'th character in
# the i'th sentence.
sentence_char_script = tf.strings.unicode_script(sentence_char_codepoint)
print(sentence_char_script)

<tf.RaggedTensor [[72, 101, 108, 108, 111, 44, 32, 119, 111, 114, 108, 100, 46], [19990, 30028, 12371, 12435, 12395, 12385, 12399]]>
<tf.RaggedTensor [[25, 25, 25, 25, 25, 0, 0, 25, 25, 25, 25, 25, 0], [17, 17, 20, 20, 20, 20, 20]]>


次に、これらのスクリプト識別子を使って、単語の境をどこに入れるべきかを決定する。これは先頭とスクリプトが変化する位置に追加される。

In [26]:
# dtype: bool; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_starts_word[i, j] is True if the j'th character in the i'th
# sentence is the start of a word.
sentence_char_starts_word = tf.concat(
    [tf.fill([sentence_char_script.nrows(), 1], True),
     tf.not_equal(sentence_char_script[:, 1:], sentence_char_script[:, :-1])],
    axis=1
)

# dtype: int64; shape: [num_words]
#
# word_starts[i] is the index of the character that starts the i'th word (in
# the flattened list of characters from all sentences).
word_starts = tf.squeeze(tf.where(sentence_char_starts_word.values), axis=1)
print(word_starts)

tf.Tensor([ 0  5  7 12 13 15], shape=(6,), dtype=int64)


これらの開始オフセットを使って、全てのバッチから得られる単語のリストを含んだ`RaggedTensor`を構築する。

In [27]:
# dtype: int32; shape: [num_words, (num_chars_per_word)]
#
# word_char_codepoint[i, j] is the codepoint for the j'th character in the
# i'th word.
word_char_codepoint = tf.RaggedTensor.from_row_starts(
    values=sentence_char_codepoint.values,
    row_starts=word_starts
)
print(word_char_codepoint)

<tf.RaggedTensor [[72, 101, 108, 108, 111], [44, 32], [119, 111, 114, 108, 100], [46], [19990, 30028], [12371, 12435, 12395, 12385, 12399]]>


結果として、単語のコードポイント`RaggedTensor`を文章にセグメントできる。

In [28]:
# dtype: int64; shape: [num_sentences]
#
# sentence_num_words[i] is the number of words in the i'th sentence.
sentence_num_words = tf.reduce_sum(
    tf.cast(sentence_char_starts_word, tf.int64),
    axis=1)

# dtype: int32; shape: [num_sentences, (num_words_per_sentence), (num_chars_per_word)]
#
# sentence_word_char_codepoint[i, j, k] is the codepoint for the k'th character
# in the j'th word in the i'th sentence.
sentence_word_char_codepoint = tf.RaggedTensor.from_row_lengths(
    values=word_char_codepoint,
    row_lengths=sentence_num_words)
print(sentence_word_char_codepoint)

<tf.RaggedTensor [[[72, 101, 108, 108, 111], [44, 32], [119, 111, 114, 108, 100], [46]], [[19990, 30028], [12371, 12435, 12395, 12385, 12399]]]>


最終結果を読みやすい形に変換できる。

In [29]:
tf.strings.unicode_encode(sentence_word_char_codepoint, 'UTF-8').to_list()

[[b'Hello', b', ', b'world', b'.'],
 [b'\xe4\xb8\x96\xe7\x95\x8c',
  b'\xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf']]